# Restricted Boltzmann Machines

## Outline
1. [What is a Boltzmann machine?](http://localhost:8888/notebooks/Introduction%20to%20Restricted%20Boltzmann%20Machines.ipynb#What-is-a-Boltzmann-machine?)
2. [What problems can we solve with one?](http://localhost:8888/notebooks/Introduction%20to%20Restricted%20Boltzmann%20Machines.ipynb#What-problems-can-we-solve-with-one?)
3. [Basic architecture](http://localhost:8888/notebooks/Introduction%20to%20Restricted%20Boltzmann%20Machines.ipynb#Basic-Architecture)
4. [*Restricted* Boltzmann machines - how and why?](http://localhost:8888/notebooks/Introduction%20to%20Restricted%20Boltzmann%20Machines.ipynb#Restricted-Boltzmann-Machines---How-and-Why?)
5. [Machinery](http://localhost:8888/notebooks/Introduction%20to%20Restricted%20Boltzmann%20Machines.ipynb#Restricted-Boltzmann-Machines---How-and-Why?)

### What is a Boltzmann machine?

A Boltzmann machine is a type of neural network. It's made out of two types of neurons: *visible* and *hidden* units. Each unit has two states - "on" or "off" (sometimes also called "activated" or "deactivated").

There is one layer of visible units, which represent the data we can feed in or read out. 

Examples:
 * Spin configurations
 * Pixel colour
 * A sentence

The visible units connect to each other and to hidden units. There may be many layers of hidden units. The connections between each neuron are *weighted*. This allows the network to encode information about features in the dataset.

Example: Spins nearby are somewhat correlated vs spins far apart are highly correlated.

Can have multiple layers of hidden units - shallower layer works as visible layer for deeper layer. Allows identification of *higher level features*.

Why not do this? We have to train the weights. We need to carefully adjust (more on this later) the weights to be able to accurately capture what's important in the data. The more layers, the more time-intensive this is.

### What problems can we solve with one?

Boltzmann machines used to solve two kinds of problems: *learning* (interesting to us) and *search*. In a *learning* problem, make small updates to parameters which will allow us to generate *new* states.

Boltzmann machines have been used to:
  * classify handwriting - what letter/digit is present in this image? ![famous MNIST problem](mnist_digits.png)
  * classify spin systems - is this the spin configuration of a para- or ferro-magnet?
  * form "deep belief networks" - many hidden layers, cheap & greedy learning, represent hierarchy of features

### Basic Architecture

We have a layer of *visible* units connected to *hidden* layer(s). 

![GBM](generalboltzmannmachine.png)

Each *unit* (visible or hidden) receives a *total input* $z_i$:

$$ z_i = b_i + \sum_j s_j W_{ij} $$

This should look familiar. $b_i$ is called the *bias* but we might recognize it as an external magnetic field. The $W_{ij}$ are the *weights* discussed above, and $s_j$ is the *state* of the $j$-th unit. Now this does look familiar! It's a classical Ising model. We might see the $W$ as $J$ - coupling constants. We will adjust their values to reproduce the training data we give the neural network. You might think of this as similar to "training" DFT to reproduce coupling constants based on observed electron density profiles.

What's interesting about this is that once the network has been trained (we have good values for the weights, which reliably reproduce the training data), we can use the trained weights to generate *new* data. This has been done and may allow us to speed up Monte Carlo simulations of spin/hardcore-boson systems and Hamiltonians we can map to them.

The total "energy" of the system is 

$$ E = - \sum_i s_i b_i - \sum_{i < j} s_i s_j W_{ij} $$

Note that each of the visible and hidden units may have a bias!

For now we will assume the states $s_i$ are *binary* - they can be 0 or 1. This is actually not required, but makes reasoning about what's happening simpler.

The probability to "turn on" a unit is determined by the so-called *logistic* or *sigmoid function*:

$$ p_i(0 \rightarrow 1) = \frac{1}{1 - \exp{(-z_i)}} $$

Where $z_i$ is the input above. Look familiar? After enough updates, the system should reach a Boltzmann distribution (thus the name).

### Why use hidden units?

We cannot directly observe their state. This lets us learn features that are not encodable in *pairwise correlations* between the visible features. This is a very important distinction from a bare Ising model we are used to!

We can learn binary features that capture higher-order relationships, because one hidden unit may be connected to many visible units (and vice-versa), in a sort of *n*-th nearest neighbor model.

### Restricted Boltzmann Machines - How and Why?

A *restricted* Boltzmann machine is a special architecture. In such a case, there are *no* connections from visible units to visible units, or from hidden units to hidden units. *All* connections (and thus weights) are from visible units to hidden units.

![RBM](generalvsrestricted.jpg)

We can represent the weights as a 2d tensor (a matrix), which we'll initialize as random numbers drawn from a uniform box distribution.

There's a bit of art to choosing the number of hidden units, which we can elide for now. Sometimes just one hidden unit will be enough, but let's hedge our bets and have a lot of them.

In [ ]:
using Distributions

number_hidden  = 32
number_visible = 64

weight_width = 1/2*sqrt(number_hidden + number_visible)
W            = rand(Uniform(-weight_width, weight_width), number_visible, number_hidden);
hidden_bias  = rand(number_hidden);
visible_bias = rand(number_visible);

#### Why use restricted vs general?
  * Given a specific visible vector (e.g. spin configuration), the hidden units are *independent* of each other
  * This means all hidden units can be updated in parallel!
  * Similarly, if we must sample visible units from hidden units, that can be done in parallel as well.

Restricted Boltzmann machines (RBMs) can learn a good weight distribution much quicker than networks with more complicated connectivity.

Now we need a way to *train* the network
Must iteratively update the weights until we get a distribution that is "good enough" (not underfitted) but still "flexible" (not overfitted).

### Machinery

Now we can discuss how to feed data into the RBM and train it.

#### Training

We want to search for the lowest cost (energy) solution.

We need to update the weights using some scheme:

$$ \Delta W_{ij} = \epsilon \left(\langle v_i h_j \rangle_{data} - \langle v_i h_j \rangle_{model} \right) $$

There are a bunch of parameters here:
  * $\epsilon$ is the *learning rate*. It controls how much the weights should "react" to disparities
  * $\langle v_i h_j \rangle_{data}$ is the association between the $i$-th visible unit and the $j$-th hidden unit sampled using the training data distribution. This is easy to compute.
  * $\langle v_i h_j \rangle_{model}$ is the association between the $i$-th visible unit and the $j$-th hidden unit sampled using the model distribution. This is not easy to compute *in an unbiased way*.


##### Computing the data sample
Sampling from the data is easy *because* the hidden units have no connections to other hidden units. This means that, given a state of all the visible units, we can find the probability that the $j$-th hidden unit is set to 1 as:

$$ p(h_j = 1 | v) = \sigma\left(b^h_j + \sum_i W_{ij}v_i\right) $$

Where $b^h_j$ are the hidden biases. $\sigma(x)$ is the *sigmoid* function defined above. $\sum_i W_{ij}v_i$ is very easy to compute - this is just a matrix-vector multiplication. These multiplications can be performed very efficiently on modern computer hardware (GPUs). Similarly, it's very easy to get an unbiased sample of the *visible* units given the hidden ones:

$$ p(v_i = 1 | h) = \sigma\left(b^v_i + \sum_j W_{ij}h_j\right) $$

##### Computing the model sample
If we are *not* given visible or hidden vectors (we'll need to compute the model association), this becomes much harder.

Often this is done with [stochastic gradient descent](http://ufldl.stanford.edu/tutorial/supervised/OptimizationStochasticGradientDescent/) (SGD), a systematized way of escaping poor local minima (what we might call "metastable states").

For other network types, like CNNs, we need to use backpropagation to update the weights efficiently. Another advantage of RBMs is that this is not necessary. We can "cheat" and use an approximation to SGD that has been shown to be very robust in practice: **constrastive divergence**.

To sample the associations in the trained model so far, we can start with a random state of the visible units and perform Gibbs sampling for many time steps. How this works:
  * Sample the hidden units from the visible units
  * Sample the visible units from the hidden units
  * Sample hidden from visible once again
... for $k$ steps.

How to pick the initial states? We pick a training vector, then "time evolve" the "system". In fact, computing the *true* gradient would still be hard, because there's a nasty term lurking which we will ignore. Instead we just compute the *difference* of two Kullback-Liebler divergences. We're using a *very* rough approximation to the true gradient that nevertheless works alright in practice. This is called $CD_k$.

In [ ]:
cd_k = 20 # set number of samples to take
ϵ    = 0.01

sigmoid(x) = 1./(1. - exp(-x))
relu(x)    = max(0, x)
# force states to be 0 or 1 using ReLU
function gibbs_sample(data, h_random, v_random)
    h_probs    = W'*data .+ hidden_bias      |> sigmoid
    h_states   = sign(-h_random .+ h_probs)  |> relu 
    v_probs    = W*h_probs .+ visible_bias   |> sigmoid 
    h_probs_1  = W'*v_probs .+ hidden_bias   |> sigmoid
    h_states_1 = sign(-h_random .+ h_probs_1)|> relu
    return h_probs, h_states, v_probs, h_probs_1, h_states_1
end

function update_parameters(input, W, hidden_bias, visible_bias, h_random, v_random)
    h_probs_0, h_states_0, v_probs_0, h_probs_1, h_states_1 = gibbs_sample(input, h_random, v_random)
    
    cd_k_step_input = v_probs_0
    for cdk_step in 2:cd_k
        h_probs, h_states, v_probs, h_probs_1, h_states_1 = gibbs_sample(cd_k_step_input, h_random, v_random)
        cd_k_step_input = v_probs
    end
    v_probs      = cd_k_step_input
    
    data_sample  = input * h_states_0'
    model_sample = v_probs * h_probs_1'
    
    W            += ϵ*(data_sample - model_sample)
    hidden_bias  += ϵ*mean(h_probs_0 - h_probs_1)
    visible_bias += ϵ*mean(input - v_probs)
    
    loss = sqrt(mean((input-v_probs).^2))
    return loss, W, hidden_bias, visible_bias
end

#### Explaining some details
We're using the ReLU (rectified linear unit) to map the probabilities to binary data. This is important to create an information bottleneck and force the RBM to "focus" on what is most important. It maps negative signs to 0 (since `sign` returns -1, 0, or 1).

The loss function tells us how poorly we are doing. Remember, `v_probs` represents the visible unit probabilities after $k$ steps of sampling, so the closer they are to the true visible unit probabilities, the lower the loss should be. The goal is to minimize this loss.

We will feed batched training samples into the training step, updating the weights and biases as we go. I've written a code to generate some simple Ising configurations using classical Monte Carlo. 

In [ ]:
# generate a training data set of Ising configurations
include("ising.jl")

num_epochs   = 100
batch_size   = 50
training_set = []
T            = 1.0
for i in 1:4
    configurations_i = generate_Ising_configurations(1./T, isqrt(number_visible))
    training_set_i   = configurations_i[10_001:end] #want equilibrated
    append!(training_set, training_set_i)
end
num_batches = div(length(training_set), batch_size)

In [ ]:
#train the network for the given set of configurations
for epoch in 1:num_epochs
    shuffle!(training_set)
    for batch_id in 1:num_batches
        batch_indices = (batch_id-1)*batch_size+1:batch_id*batch_size
        #make a 2D matrix of examples
        batch    = hcat(training_set[batch_indices]...)
        #add noise to the bias
        h_random = rand(number_hidden, batch_size)
        v_random = rand(batch_size, number_visible)
        #update weights and biases
        loss, W, hidden_bias, visible_bias = update_parameters(batch, W, hidden_bias, visible_bias, h_random, v_random)
    end
end

![weights](t_weights.png)

### Further directions

Here we've used binary data, but there are other choices. In fact, we can use any distribution for the neurons from the "exponential family", including:
 * Binary
 * Gaussian
 * Poisson
 * Binomial
 * ReLU

This may allow us to generalize the RBMs to other problems. We can also use a pre-trained RBM to look at other, similar physics problems.